In [22]:
import pandas as pd
import numpy as np 

In [171]:
df = pd.read_excel(
    "https://www.eia.gov/dnav/ng/xls/NG_PRI_SUM_A_EPG0_PEU_DMCF_M.xls", 
    sheet_name="Data 1",
    skiprows=2,
    index_col=0)
df

,U.S. Natural Gas Electric Power Price (Dollars per Thousand Cubic Feet),Alabama Natural Gas Price Sold to Electric Power Consumers (Dollars per Thousand Cubic Feet),Alaska Natural Gas Price Sold to Electric Power Consumers (Dollars per Thousand Cubic Feet),Arizona Natural Gas Price Sold to Electric Power Consumers (Dollars per Thousand Cubic Feet),Arkansas Natural Gas Price Sold to Electric Power Consumers (Dollars per Thousand Cubic Feet),California Natural Gas Price Sold to Electric Power Consumers (Dollars per Thousand Cubic Feet),Colorado Natural Gas Price Sold to Electric Power Consumers (Dollars per Thousand Cubic Feet),Connecticut Natural Gas Price Sold to Electric Power Consumers (Dollars per Thousand Cubic Feet),Delaware Natural Gas Price Sold to Electric Power Consumers (Dollars per Thousand Cubic Feet),District of Columbia Natural Gas Price Sold to Electric Power Consumers (Dollars per Thousand Cubic Feet),...,South Dakota Natural Gas Price Sold to Electric Power Consumers (Dollars per Thousand Cubic Feet),Tennessee Natural Gas Price Sold to Electric Power Consumers (Dollars per Thousand Cubic Feet),Texas Natural Gas Price Sold to Electric Power Consumers (Dollars per Thousand Cubic Feet),Utah Natural Gas Price Sold to Electric Power Consumers (Dollars per Thousand Cubic Feet),Vermont Natural Gas Price Sold to Electric Power Consumers (Dollars per Thousand Cubic Feet),Virginia Natural Gas Price Sold to Electric Power Consumers (Dollars per Thousand Cubic Feet),Washington Natural Gas Price Sold to Electric Power Consumers (Dollars per Thousand Cubic Feet),West Virginia Natural Gas Price Sold to Electric Power Consumers (Dollars per Thousand Cubic Feet),Wisconsin Natural Gas Price Sold to Electric Power Consumers (Dollars per Thousand Cubic Feet),Wyoming Natural Gas Price Sold to Electric Power Consumers (Dollars per Thousand Cubic Feet)
Date,,,,,,,,,,,,,,,,,,,,,
2002-01-15,3.10,NaN,NaN,2.62,NaN,3.42,2.86,2.89,NaN,NaN,...,NaN,NaN,2.65,NaN,3.54,NaN,2.72,3.23,NaN,7.21
2002-02-15,2.86,NaN,2.57,2.54,NaN,3.23,2.56,2.75,NaN,NaN,...,NaN,NaN,2.36,9.98,2.73,NaN,NaN,3.01,2.45,5.09
2002-03-15,3.37,NaN,2.77,3.28,NaN,3.71,2.92,3.66,NaN,NaN,...,NaN,NaN,2.96,NaN,3.13,NaN,NaN,3.41,2.74,4.43
2002-04-15,3.80,NaN,2.46,3.31,NaN,4.01,2.91,4.17,NaN,NaN,...,NaN,NaN,3.57,NaN,NaN,NaN,NaN,4.29,3.55,3.91
2002-05-15,3.78,NaN,2.38,3.00,NaN,3.84,2.71,4.05,NaN,NaN,...,NaN,NaN,3.52,NaN,NaN,NaN,NaN,4.44,3.83,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-15,4.64,3.01,8.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.84,3.53,2.62,11.69,NaN,5.53,NaN,NaN,NaN,NaN
2023-03-15,3.51,NaN,8.05,NaN,NaN,NaN,NaN,4.45,NaN,NaN,...,2.82,2.89,2.51,6.08,NaN,4.06,NaN,NaN,NaN,NaN
2023-04-15,2.81,NaN,8.15,2.87,NaN,NaN,NaN,2.07,NaN,NaN,...,2.56,2.45,2.17,4.71,NaN,3.84,NaN,NaN,NaN,NaN


In [74]:
df = pd.read_excel(
    "https://www.eia.gov/dnav/ng/xls/NG_PRI_SUM_A_EPG0_PEU_DMCF_M.xls", 
    sheet_name="Data 1",
    skiprows=2,
    index_col=0)
df = df
fill_values = {x:df["U.S. Natural Gas Electric Power Price (Dollars per Thousand Cubic Feet)"] for x in df.columns}
df = df.fillna(fill_values).reset_index()
df = df.melt(id_vars="Date", var_name="series-description")
df["units"] = df["series-description"].map(lambda x: x.split("(")[1].split(")")[0].strip())
df["units"] = df["units"].map(lambda x: "$/MCF" if x == "Dollars per Thousand Cubic Feet" else x)
df["state"] = df["series-description"].map(lambda x: x.split("Natural Gas")[0].strip())
df["period"] = pd.to_datetime(df["Date"])
df = df.set_index("period").drop(columns=["Date"])
df = df[df.index.year == 2022]
df

,series-description,value,units,state
period,,,,
2022-01-15,U.S. Natural Gas Electric Power Price (Dollars...,6.97,$/MCF,U.S.
2022-02-15,U.S. Natural Gas Electric Power Price (Dollars...,6.26,$/MCF,U.S.
2022-03-15,U.S. Natural Gas Electric Power Price (Dollars...,5.32,$/MCF,U.S.
2022-04-15,U.S. Natural Gas Electric Power Price (Dollars...,6.45,$/MCF,U.S.
2022-05-15,U.S. Natural Gas Electric Power Price (Dollars...,7.79,$/MCF,U.S.
...,...,...,...,...
2022-08-15,Wyoming Natural Gas Price Sold to Electric Pow...,9.33,$/MCF,Wyoming
2022-09-15,Wyoming Natural Gas Price Sold to Electric Pow...,9.48,$/MCF,Wyoming
2022-10-15,Wyoming Natural Gas Price Sold to Electric Pow...,6.03,$/MCF,Wyoming


In [170]:
df = pd.read_csv("./../resources/costs/ng_electric_power_price.csv").set_index("period")
df["state"] = df["series-description"].map(lambda x: x.split("Natural Gas")[0].strip())
df["value"] = df["value"].fillna(np.nan)
usa_average = df[df.state == "U.S."].to_dict()["value"]

df = df.reset_index()
df["use_average"] = ~pd.notna(df["value"]) # not sure why this cant be built into the lambda function 
df["value"] = df.apply(lambda x: x["value"] if not x["use_average"] else usa_average[x["period"]], axis=1)
df = df.drop(columns=["use_average"])
df

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units,state
0,2022-12-01,STX,TEXAS,EPG0,Natural Gas,PEU,Electric Power Price,N3045TX3,Texas Natural Gas Price Sold to Electric Power...,5.34,$/MCF,Texas
1,2022-12-01,NUS,U.S.,EPG0,Natural Gas,PEU,Electric Power Price,N3045US3,U.S. Natural Gas Electric Power Price (Dollars...,9.53,$/MCF,U.S.
2,2022-12-01,SME,USA-ME,EPG0,Natural Gas,PEU,Electric Power Price,N3045ME3,Maine Natural Gas Price Sold to Electric Power...,9.53,$/MCF,Maine
3,2022-12-01,SWI,USA-WI,EPG0,Natural Gas,PEU,Electric Power Price,N3045WI3,Wisconsin Natural Gas Price Sold to Electric P...,6.32,$/MCF,Wisconsin
4,2022-12-01,SOK,USA-OK,EPG0,Natural Gas,PEU,Electric Power Price,N3045OK3,Oklahoma Natural Gas Price Sold to Electric Po...,9.53,$/MCF,Oklahoma
...,...,...,...,...,...,...,...,...,...,...,...,...
595,2022-01-01,SOH,OHIO,EPG0,Natural Gas,PEU,Electric Power Price,N3045OH3,Ohio Natural Gas Price Sold to Electric Power ...,4.26,$/MCF,Ohio
596,2022-01-01,SID,USA-ID,EPG0,Natural Gas,PEU,Electric Power Price,N3045ID3,Idaho Natural Gas Price Sold to Electric Power...,6.91,$/MCF,Idaho
597,2022-01-01,SVT,USA-VT,EPG0,Natural Gas,PEU,Electric Power Price,N3045VT3,Vermont Natural Gas Price Sold to Electric Pow...,6.97,$/MCF,Vermont
598,2022-01-01,SSD,USA-SD,EPG0,Natural Gas,PEU,Electric Power Price,N3045SD3,South Dakota Natural Gas Price Sold to Electri...,5.75,$/MCF,South Dakota


In [166]:
usa_average

{1: 9.53,
 72: 5.96,
 102: 6.03,
 157: 8.46,
 202: 9.33,
 297: 7.76,
 340: 8.23,
 368: 7.79,
 428: 6.45,
 475: 5.32,
 529: 6.26,
 563: 6.97}

In [97]:
import pandas as pd

# Sample DataFrame
data = {'A': [1, 2, 3],
        'B': [4, 5, 6],
        'C': [7, 8, 9]}

df = pd.DataFrame(data)

# Define a custom function to apply across rows
def custom_function(row):
    # You can access row elements like row['A'], row['B'], etc.
    return row['A'] + row['B'] + row['C']

# Apply the custom function to each row
result = df.apply(custom_function, axis=1)

# Add the result as a new column in the DataFrame
df['Result'] = result

# Display the modified DataFrame
print(df)

   A  B  C  Result
0  1  4  7      12
1  2  5  8      15
2  3  6  9      18


In [44]:
import pandas as pd
import numpy as np

# Create a sample DataFrame with NaN values
data = {'A': [1, np.nan, 3, 4, np.nan],
        'B': [5, 6, np.nan, 8, 9],
        'C': [10, 11, 12, 13, 14]}

df = pd.DataFrame(data)

# Specify the column you want to use as the source for replacement
source_column = 'B'  # Change this to the desired column

# Use the `fillna` method to replace all NaN values in the entire DataFrame
df.fillna(df[source_column], inplace=True)

# Display the modified DataFrame
print(df)

     A    B   C
0  1.0  5.0  10
1  NaN  6.0  11
2  3.0  NaN  12
3  4.0  8.0  13
4  NaN  9.0  14
